# Basic Query Construction

In this notebook, we'll explore how to construct basic queries using individual match directives. We'll cover each type of match directive and show how to configure them for different use cases.

## Setup

First, let's import the necessary modules:

In [ ]:
import json
from elastictoolkit.queryutils.builder.matchdirective import (
    ConstMatchDirective,
    TextMatchDirective,
    RangeMatchDirective,
    FieldExistsDirective,
    ScriptMatchDirective,
    QueryStringMatchDirective
)
from elastictoolkit.queryutils.consts import FieldMatchType, MatchMode
from elastictoolkit.queryutils.types import NestedField

# Helper function to print queries as formatted JSON
def print_query(directive):
    query = directive.to_dsl()
    print(json.dumps(query.to_query(), indent=2))

## ConstMatchDirective

The `ConstMatchDirective` is used for exact matching. It generates a `term` query for single values or a `terms` query for multiple values. This is ideal for matching structured data like categories, tags, or IDs.

In [ ]:
# Create a simple ConstMatchDirective
directive = ConstMatchDirective(rule=FieldMatchType.ANY)
directive.set_field("category")
directive.set_values("electronics")
directive.set_match_params({})

print_query(directive)

### Multiple Values

You can match against multiple values using the `rule` parameter to control how they're combined:

In [ ]:
# Match ANY of the values (OR logic)
directive = ConstMatchDirective(rule=FieldMatchType.ANY)
directive.set_field("category")
directive.set_values("electronics", "computers", "accessories")
directive.set_match_params({})

print_query(directive)

In [ ]:
# Match ALL of the values (AND logic)
directive = ConstMatchDirective(rule=FieldMatchType.ALL)
directive.set_field("tags")
directive.set_values("wireless", "bluetooth", "headphones")
directive.set_match_params({})

print_query(directive)

### Multiple Fields

You can also match against multiple fields:

In [ ]:
directive = ConstMatchDirective(rule=FieldMatchType.ANY)
directive.set_field("category", "subcategory")
directive.set_values("electronics")
directive.set_match_params({})

print_query(directive)

### Exclusion Mode

You can use the `mode` parameter to exclude documents that match:

In [ ]:
directive = ConstMatchDirective(rule=FieldMatchType.ANY, mode=MatchMode.EXCLUDE)
directive.set_field("category")
directive.set_values("discontinued")
directive.set_match_params({})

print_query(directive)

## TextMatchDirective

The `TextMatchDirective` is used for full-text search. It generates a `match` query for single fields or a `multi_match` query for multiple fields. This is ideal for searching text content like titles, descriptions, or content.

In [ ]:
directive = TextMatchDirective(rule=FieldMatchType.ANY)
directive.set_field("title", "description")
directive.set_values("wireless headphones")
directive.set_match_params({})

print_query(directive)

### Advanced Text Matching

You can configure additional parameters for text matching:

In [ ]:
directive = TextMatchDirective(rule=FieldMatchType.ANY)
directive.set_field("title", "description")
directive.set_values("wireless headphones")
directive.set_match_params({})
directive.set_match_query_extra_args(
    fuzziness="AUTO",
    operator="and",
    minimum_should_match="75%"
)

print_query(directive)

## RangeMatchDirective

The `RangeMatchDirective` is used for range queries. It generates a `range` query that matches documents with field values within a specified range.

In [ ]:
directive = RangeMatchDirective()
directive.set_field("price")
directive.set_values(gt=50, lte=200)
directive.set_match_params({})

print_query(directive)

### Date Ranges

Range queries are particularly useful for date ranges:

In [ ]:
directive = RangeMatchDirective()
directive.set_field("created_date")
directive.set_values(gte="2023-01-01", lt="2024-01-01")
directive.set_match_params({})

print_query(directive)

## FieldExistsDirective

The `FieldExistsDirective` is used to check if a field exists in a document. It generates an `exists` query.

In [ ]:
directive = FieldExistsDirective()
directive.set_field("thumbnail")
directive.set_match_params({})

print_query(directive)

### Checking for Missing Fields

You can also check for missing fields using the `mode` parameter:

In [ ]:
directive = FieldExistsDirective(mode=MatchMode.EXCLUDE)
directive.set_field("thumbnail")
directive.set_match_params({})

print_query(directive)

## ScriptMatchDirective

The `ScriptMatchDirective` is used to match documents using a script. It generates a `script` query that allows for complex matching logic.

In [ ]:
directive = ScriptMatchDirective(
    script="doc['price'].value < doc['msrp'].value * 0.75"
)
directive.set_match_params({})

print_query(directive)

### Scripts with Parameters

You can use parameters in your scripts for better reusability:

In [ ]:
directive = ScriptMatchDirective(script="doc['price'].value < doc['msrp'].value * params.discount_factor")
directive.set_values(discount_factor=0.75) # All the kwargs passed here will end up in script-params
directive.set_match_params({})

print_query(directive)

## QueryStringMatchDirective

The `QueryStringMatchDirective` is used for complex text queries using Elasticsearch's query string syntax. It generates a `query_string` query.

In [ ]:
directive = QueryStringMatchDirective(rule=FieldMatchType.ANY)
directive.set_field("title", "description")
directive.set_values("wireless AND (headphones OR earbuds) -wired")
directive.set_match_params({})

print_query(directive)

### Advanced Query String Options

You can configure additional parameters for query string matching:

In [ ]:
directive = QueryStringMatchDirective(rule=FieldMatchType.ANY)
directive.set_field("title", "description")
directive.set_values("wireless AND (headphones OR earbuds) -wired")
directive.set_match_params({})
directive.set_match_query_extra_args(
    default_operator="AND",
    fuzziness="AUTO",
    boost=2.0
)

print_query(directive)

## Working with Nested Fields

Elasticsearch supports nested fields for complex document structures. The toolkit provides the `NestedField` class to work with nested fields:

In [ ]:
directive = ConstMatchDirective(rule=FieldMatchType.ANY)
directive.set_field(
    NestedField(nested_path="reviews", field_name="reviews.rating")
)
directive.set_values(5)
directive.set_match_params({})

print_query(directive)

### Multiple Nested Fields

You can combine multiple nested fields in a single directive:

In [ ]:
directive = ConstMatchDirective(rule=FieldMatchType.ANY)
directive.set_field(
    NestedField(nested_path="reviews", field_name="reviews.rating"),
    NestedField(nested_path="variants", field_name="variants.in_stock")
)
directive.set_values(True)
directive.set_match_params({})

print_query(directive)

## Dynamic Values with Match Parameters

One of the powerful features of the toolkit is the ability to use dynamic values through match parameters. This allows you to create reusable directives that adapt to runtime parameters.

In [ ]:
from elastictoolkit.queryutils.builder.directivevaluemapper import DirectiveValueMapper
from elastictoolkit.queryutils.types import FieldValue

# Define a value mapper
class ProductValueMapper(DirectiveValueMapper):
    price_range = FieldValue(
        fields=["price"],
        values_map={
            "gte": "match_params.min_price",
            "lte": "match_params.max_price"
        }
    )

# Create a directive
directive = RangeMatchDirective()
directive.set_field("price")

# Set match parameters
match_params = {"min_price": 50, "max_price": 200}
directive.set_match_params(match_params)

# Use the value mapper to resolve values
value_mapper = ProductValueMapper()
attr_field_mapping = value_mapper.get_field_value("price_range")
directive.set_values(**attr_field_mapping.values_map)

print_query(directive)

## Combining Multiple Directives

While individual directives are powerful, the real strength of the toolkit comes from combining multiple directives. In the next notebook, we'll explore how to use directive engines to orchestrate multiple directives.

## Summary

In this notebook, we've explored the basic query construction using individual match directives:

- **ConstMatchDirective**: For exact matching
- **TextMatchDirective**: For full-text search
- **RangeMatchDirective**: For range queries
- **FieldExistsDirective**: For checking if fields exist
- **ScriptMatchDirective**: For script-based matching
- **QueryStringMatchDirective**: For complex text queries

We've also seen how to work with nested fields and dynamic values using match parameters.

In the next notebook, we'll explore how to use directive engines to orchestrate multiple directives and build more complex queries.